In [108]:
import csv
import json
import os
import pandas as pd

Load json files

In [100]:
neurofinder_df = pd.read_json('neurofinder-results.json')
neurofinder_df.loc[:,'contents'] = neurofinder_df.loc[:,'results']
del neurofinder_df['results']

spikefinder_df = pd.read_json('spikefinder-results.json')

'Flatten' json entries to generate rows for each `algorithm x dataset x result`

In [115]:
def reformat_df(df):
    out_df = pd.DataFrame(columns=['algo__id', 'algo_contact', 'algo___v', 'algo_timestamp', 
                                   'algo_repository', 'algo_name', 'algo_algorithm', 
                                   'dset_dataset', 'dset__id', 'label__id', 'label_label', 'label_value'])

    quick_filter = lambda dct, tag, excl: {tag + key:value for key, value in dct.items() if not key in excl}

    for algo in spikefinder_df.iterrows():
        algo_dict = quick_filter(dict(algo[1]), 'algo_', ['contents',])

        for dset in algo[1]['contents']:
            dset_dict = quick_filter(dset, 'dset_', ['scores',])

            for label in dset['scores']:
                label_dict = quick_filter(label, 'label_', [])    

                row_dict = {}
                row_dict.update(algo_dict)
                row_dict.update(dset_dict)
                row_dict.update(label_dict)

                out_df = pd.concat([out_df, pd.DataFrame.from_dict([row_dict,])])
    
    out_df = out_df.reset_index()
    del out_df['index']
    
    return out_df

In [116]:
neurofinder_df_R = reformat_df(neurofinder_df)
neurofinder_df_R.to_csv('neurofinder.csv')

In [117]:
spikefinder_df_R = reformat_df(spikefinder_df)
spikefinder_df_R.to_csv('spikefinder.csv')